# Assignment 11

## Mounting the Drive

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [17]:
DataFolder_Path = "/content/gdrive/MyDrive/EETBS /Data"

## Importing Modules

In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [19]:
ConsumptionFileName= "consumption_5545.csv"
TemperatureFileName= "Austin_weather_2014.csv"
IrradianceFileName= "irradiance_2014_gen.csv"

## Files

In [20]:
ConsumptionFilePath = os.path.join(DataFolder_Path,ConsumptionFileName)
TemperatureFilePath  = os.path.join(DataFolder_Path,TemperatureFileName)
IrradianceFilePath = os.path.join(DataFolder_Path,IrradianceFileName)

In [38]:
DF_consumption = pd.read_csv(ConsumptionFilePath,sep=",", index_col=0, parse_dates=[0])

DF_weather = pd.read_csv(TemperatureFilePath,sep=";",index_col = ["localhour"],parse_dates=["localhour"],
                         date_parser= lambda col: pd.to_datetime(col, utc=True).tz_convert("US/Central").tz_localize(None)) 
DF_temperature = DF_weather[["temperature"]]

DF_irradiance = pd.read_csv(IrradianceFilePath,sep=";", index_col=["localhour"], parse_dates=["localhour"])
DF_irradiance = DF_irradiance[["gen"]]
DF_irradiance.loc[DF_irradiance["gen"]<0,"gen"]=0

In [39]:
DF_joined  = DF_consumption.join([DF_temperature,DF_irradiance])

DF_joined = DF_joined.rename(columns ={"air conditioner_5545":"Consumption [W]","temperature":"Temperature [°K]","gen":"Irradiance [??]"})

In [23]:
DF_joined.dropna(inplace = True)

In [24]:
Joined_DF_FileName= "joined_DF.csv"
RamosFolder_Path = "/content/gdrive/MyDrive/EETBS /Data/Ramos"
Joined_DF_FilePath = os.path.join(RamosFolder_Path,Joined_DF_FileName)
DF_joined.to_csv(Joined_DF_FilePath)

In [29]:
DF_mod = DF_joined.copy() 

## Task

### Functions

In [26]:
def laggedColumnCreator(df,ColumnName,LagStart,LagEnd,LagInterval):
  for i in list(range(LagStart,LagEnd+1,LagInterval)):
    NewColumnName = ColumnName + " -" + str(i) + "h"
    df[NewColumnName]= df[ColumnName].shift(i)
    df.dropna(inplace=True) #This applies it to the dataframe; if not you should use df=df.dropna()
  return df

In [37]:
def Function(df): #This function assumes the index to be the time stamp
  df["Hours"] = df.index.hour
  df["sin(hour)"] = np.sin(df["Hours"])
  df["cos(hour)"] = np.cos(df["Hours"])
  df["Day of the Week"] = df.index.isocalendar().day # Monday is 1
  df["Month"] = df.index.month
  df["Week of the Year"] = df.index.isocalendar().week # I don't understand why but this function considers the last three days of the years as part of the first week of the following year
  df["Weekend Flag"] = np.where(df.index.dayofweek<5,0,1)
  df.dropna(inplace=True)
  return df

### Lags of Consumption Temperature and Irradiance

In [32]:
DF_mod = laggedColumnCreator(DF_mod,"Consumption [W]",1,24,1)
DF_mod = laggedColumnCreator(DF_mod,"Irradiance [??]",3,6,1)

### Adding Time-Related Features

In [40]:
DF_mod = Function(DF_mod)

In [42]:
DF_mod.tail(60)

,Consumption [W],Temperature [°K],Irradiance [??],Consumption [W] -1h,Consumption [W] -2h,Consumption [W] -3h,Consumption [W] -4h,Consumption [W] -5h,Consumption [W] -6h,Consumption [W] -7h,Consumption [W] -8h,Consumption [W] -9h,Consumption [W] -10h,Consumption [W] -11h,Consumption [W] -12h,Consumption [W] -13h,Consumption [W] -14h,Consumption [W] -15h,Consumption [W] -16h,Consumption [W] -17h,Consumption [W] -18h,Consumption [W] -19h,Consumption [W] -20h,Consumption [W] -21h,Consumption [W] -22h,Consumption [W] -23h,Consumption [W] -24h,Irradiance [??] -3h,Irradiance [??] -4h,Irradiance [??] -5h,Irradiance [??] -6h,Hours,sin(hour),cos(hour),Day of the Week,Month,Week of the Year,Weekend Flag
2014-12-28 13:00:00,0.0,43.27,1.021800,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.353850,0.125617,0.015533,0.000000,13,0.420167,0.907447,7,12,52,1
2014-12-28 14:00:00,0.0,44.24,1.099583,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.115983,0.353850,0.125617,0.015533,14,0.990607,0.136737,7,12,52,1
2014-12-28 15:00:00,0.0,44.94,0.875217,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.206067,1.115983,0.353850,0.125617,15,0.650288,-0.759688,7,12,52,1
2014-12-28 16:00:00,0.0,45.03,1.388950,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.021800,1.206067,1.115983,0.353850,16,-0.287903,-0.957659,7,12,52,1
2014-12-28 17:00:00,0.0,45.94,0.221383,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.099583,1.021800,1.206067,1.115983,17,-0.961397,-0.275163,7,12,52,1
2014-12-28 18:00:00,0.0,42.84,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.875217,1.099583,1.021800,1.206067,18,-0.750987,0.660317,7,12,52,1
2014-12-28 19:00:00,0.0,39.32,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.388950,0.875217,1.099583,1.021800,19,0.149877,0.988705,7,12,52,1
2014-12-28 20:00:00,0.0,37.55,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.221383,1.388950,0.875217,1.099583,20,0.912945,0.408082,7,12,52,1
2014-12-28 21:00:00,0.0,36.39,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.221383,1.388950,0.875217,21,0.836656,-0.547729,7,12,52,1
2014-12-28 22:00:00,0.0,35.23,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.221383,1.388950,22,-0.008851,-0.999961,7,12,52,1
